In [ ]:
# Instalar las dependencias necesarias para el script
!pip install google-search-results openpyxl

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=6add454c483b0655962cadf7941584734ddc03fc44647ec33f1df254a8ae9390
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [ ]:
import re # Libreria regex para expresiones regulares
import pandas as pd
import os # Libreria para obtener variable de entorno
from serpapi import GoogleSearch # Cliente de SerpAPI para realizar busquedas en google y obtener resultado en JSON
from openpyxl import Workbook # Libreria para crear y manipular archivos excel

"""
Estaré guardando los puestos en un libro de excel, este script va a buscar individualmente los puestos de trabajo.
Estos tendrán el nombre del puesto y el pais de este mismo. Van a estar ubicados en el apartado de files que proporciona Google Colab.
Estoy usando el servicio SerpAPI es cual permite obtener resutlados de varios motores de busqueda en formato JSON.
Seleccione este servicio debido a que con beautifulsoup, algunas paginas como indeed me bloqueaban el acceso del request, y este servicio ya se encarga de ese detalle.
Esta la documentacion de la API para el caso de los empleos y busqueda con google search:
    https://serpapi.com/google-jobs-results
    https://serpapi.com/search-api
"""
# Consantes globales
# Configuro la llave de la API  de serpapi y los elementos necesarios para realizar la busqueda de puestos de trabajo
# Todas estas variables seran constantes debido a que no deben cambiar durante la ejecución
SERPAPI_KEY = os.getenv("SERPAPI_KEY")
POSITION = "front end developer"
LOCATION = "Mexico"
COUNTRY_CODE = "mx"
# para los puestos en estados unidos
#POSITION = "data engineering"
#LOCATION = "United States"
#COUNTRY_CODE = "us"
MAX_RESULTS = 15
EXCEL_FILENAME = f"resultados_{POSITION}_{LOCATION}.xlsx" # El nombre del archivo tiene el nombre de la posicion y el pais de donde viene

# Esta constante es una lista que contiene expresiones regulares para los salarios, estos tienen diferentes formatos como vienen abajo
SALARY_FORMAT = [
    re.compile(r'\$[\d,]+(?:\.\d{2})?(?:\s*-\s*\$[\d,]+(?:\.\d{2})?)?'),  # Ej: $10,000 - $20,000
    re.compile(r'[\d,]+(?:\.\d{2})?\s*(?:USD|MXN|pesos?|dollars?)', re.IGNORECASE),  # Ej: 15,000 MXN
    re.compile(r'[\d,]+(?:\.\d{2})?(?:k|K)'),  # Ej: 20k
]

# Esta constante es una lista que contiene palabras clave del stack tecnologico requerido para los puestos, hice una pequeña investigacion y con ayuda
# de la IA, identifque algunas de las tecnologias mas recurrentes para estos puestos de trabajo. Estos seran mostrados como los technical skills en el excel del puesto.
KEYWORDS_TECHNICAL = [
    'uml', 'java', 'python', 'c#', 'c++', '.net', 'spring', 'sql', 'aws', 'azure', 'css',
    'html', 'javascript', 'typescript', 'react', 'angular', 'laravel', 'vue', 'node.js',
    'mongodb', 'postgresql', 'mysql', 'sass', 'tailwind', 'bootstrap', 'git', 'rest api',
    'graphql', 'gitlab', 'go', 'php', 'django', 'flask', 'junit', 'ci/cd', 'docker',
    'github', 'postman', 'pytest', 'linux', 'kubernetes', 'jenkins', 'github actions',
    'gitlab ci/cd', 'gcp', 'r', 'cassandra', 'pandas', 'pytorch', 'tensorflow', 'spark',
    'jira', 'click up', 'numpy', 'power bi', 'tableau', 'figma', 'confluence', 'swift', 'flutter',
    'rust', 'ruby', 'react native'
]

# Esta constante es una lista de diccionarios donde como la key es el nombre de la habilidad y el valor son palabras relacionadas a la habilidad.
# De igual forma como las technical skills, realize una pequeña busqueda de cuales eran las soft skills mas recurrentes, estas de igual forma seran vistas como las soft skill en el excel.
KEYWORDS_SOFT = [
        {"skill": "comunicacion", "keywords": ["comunicacion", "communication", "habilidades de comunicacion", "interpersonal skills"]},
        {"skill": "trabajo en equipo", "keywords": ["trabajo en equipo", "teamwork", "colaboracion", "team player", "cooperation"]},
        {"skill": "liderazgo", "keywords": ["liderazgo", "leadership", "mentor", "coaching", "management"]},
        {"skill": "resolucion de problemas", "keywords": ["resolucion de problemas", "problem solving", "analytical thinking", "troubleshooting"]},
        {"skill": "pensamiento critico", "keywords": ["pensamiento critico", "critical thinking", "logical reasoning"]},
        {"skill": "adaptabilidad", "keywords": ["adaptabilidad", "adaptability", "flexible", "resilience", "agile"]},
        {"skill": "gestion del tiempo", "keywords": ["gestion del tiempo", "time management", "prioritization"]},
        {"skill": "creatividad", "keywords": ["creatividad", "creativity", "innovative", "idea generation"]},
        {"skill": "gestion de proyectos", "keywords": ["gestion de proyectos", "project management", "planning", "organization"]},
        {"skill": "inteligencia emocional", "keywords": ["inteligencia emocional", "emotional intelligence", "self-awareness", "empathy"]},
        {"skill": "negociacion", "keywords": ["negociacion", "negotiation", "persuasion", "influencing"]},
        {"skill": "orientacion a resultados", "keywords": ["orientacion a resultados", "results-driven", "goal oriented", "achievement"]},
        {"skill": "proactividad", "keywords": ["proactividad", "initiative", "self-starter"]},
        {"skill": "toma de decisiones", "keywords": ["toma de decisiones", "decision making", "judgement"]},
        {"skill": "gestion del estres", "keywords": ["gestion del estres", "stress management", "calm under pressure"]},
        {"skill": "aprendizaje continuo", "keywords": ["aprendizaje continuo", "continuous learning", "self learning", "growth mindset"]},
        {"skill": "presentacion", "keywords": ["presentacion", "public speaking", "presenting", "pitching"]},
        {"skill": "multitarea", "keywords": ["multitarea", "multitasking", "task switching"]},
]

# Funciones auxiliares
def save_to_excel(jobs_data):
  """
  Esta funcion guarda la lista de trabajos en un archivo excel.
  Args:
    jobs_data: Una lista de diccionarios que contiene los datos de los trabajos.
  Returns:
    booL: True si se pudo guardar el archivo, False en caso contrario.
  """
  if not jobs_data:
    print("No hay datos para guaradr en el excel.")
    return False

  df = pd.DataFrame(jobs_data)
  df.to_excel(EXCEL_FILENAME, index=False)
  print(f"Los datos se han guardado en el archivo: {EXCEL_FILENAME}")
  return True


def extract_skills(text, technical_skills, soft_skills):
  """
  Esta funcion extrae las technical y soft skills de un texto dado.
  Args:
    text (str): Texto de descripcion del puesto
    technical_skills (list): Lista de palabras clave de las technical skills
    soft_skills (list): Lista de diccionarios con las soft skills y sus palabras clave
  Returns:
    tuple: Una tupla con dos listas, la primera contiene las technical skills y la segunda contiene las soft skills.
  """
  # En caso de que el texto este vacio, quiere decir que no se pueden detectar skills y retorna 2 listas vacias
  if not text:
    return [], []
  text = text.lower() # Normaliza el texto recibo a minusculas para evitar problemas de letras mayusculas y minisculas al comparar

  # A continuacion creo una lista para cada tipo de habilidad
  match_technical_skill = [
      skill.title() # Convierte la primera letra de las habilidades en mayuscula
      for skill in technical_skills # recorre las habilidades que existen en la lista de las technical skills
      if skill in text # luego verifica si esa palabra se encuentra en la descripcion del puesto
  ]
  # Aqui es igual a las technical skills, lo que cambia es que es una lista de diccionarios
  match_soft_skill = [
      s["skill"].title() # Creo una variable que representaun diccionario de la lista de las soft skills, de igual forma su primera letra sera mayuscula
      for s in soft_skills # recorro las habilidades que estan en las softskills
      if any(p in text for p in s["keywords"]) # y luego uso el any() para comprobar si alguna palabra clave (que es la variable "p") de las palabras clave de cada diccionario s
  ]

  return match_technical_skill[:15],  match_soft_skill[:15] # devuelve 2 listas que contienen 15 habilidades blandas y tecnicas

def extract_salary(text):
  """
  Esta funcion extrae de un texto el salario segun sean los formatos definidos
  Args:
    text (str): Texto donde puede estar el salario
  Returns:
    str: El salario extraido del texto o "N/A" si no se encuentra.
  """
  if not text: # Si no encuentra algun texto recibido, solo devuelve un N/A
    return "N/A"

  text = str(text).strip() # convierto el texto en string y quito los espacios en los extremos
  for format in SALARY_FORMAT: # recorro los patrones de las expresiones definidas en SALARY_FORMAT
    # busco el patron del texto, si encuentra algun match o coincidencia, devuelve el fragmento que coincidio
    match = format.search(text)
    if match:
      return match.group(0) # y si no encuentra nada, solo devuelve los primeros 100 caracteres como fallback
  return text[:100]

def get_job_urls(job_item, COUNTRY_CODE):
  """
  Esta funcion obtiene el URL del puesto desde los campos de apply_options o related links.
  Args:
    job_item (dict): Diccionario con la informacion del trabajo
    COUNTRY_CODE (str): Codigo del pais de donde viene el puesto para filtrar los links invalidos de Google
  Returns:
    str: URL del puesto o "El URL de este puesto no esta disponible." si no se encuentra.
  """
  def is_valid_link(url): # Cree una funcion interna para validar que el link no ea una busqueda de Google
    google_patterns = [
        f'https://www.google.com/search?',
        f'https://google.com{COUNTRY_CODE}/search'
    ]
    return not any(pat in url for pat in google_patterns) # Devuelve True si ninguno de esos patrones aparece en la URL.

  # Busca primero en los apply_options
  for opc in job_item.get('apply_options', []): # itera sobre las opciones que tiene SerpAPI para aplicar
    if isinstance(opc, dict): # Asegura que el elemento sea un diccionario
      url = opc.get('link') # Extrae el campo del link
      if url and is_valid_link(url): # Si existe un link y pasa la validacion, lo devuelve como URL final de la oferta de trabajo
        return url

  # En caso de que no encuentre el link, lo buscara en los related links
  # El proceso es el mismo de las apply options
  for opc in job_item.get('related_links', []):
    if isinstance(opc, dict):
      url = opc.get('link')
      if url and is_valid_link(url):
        return url

  return "El URL de este puesto no esta disponible." # En caso de que no se encuentre la URL valida, muestro un mensaje informativo

def get_lenguage(text):
  """
  Esta funcion detecta si se quiere ingles o ser bilingue en un texto dado.
  Args:
    text (str): Texto de descripcion del puesto
  Returns:
    str: "Si" si se quiere ingles o ser bilingue, "N/A" en caso contrario.
  """
  # Devuelvo un si se detecta alguna palabra que involucre el ingles o bilingue
  return "Si" if any(
      w in text for w in ['ingles', 'english', 'bilingue', 'bilingual'] # busca en cualquiera de estas variantes
  ) else "N/A" # en caso de que no lo encuentre, devuelvo un N/A

def get_academic_profile(text):
  """
  Esta funcion obtiene el perfil academico de un texto dado.
  Args:
    text (str): Texto de descripcion del puesto
  Returns:
    str: Perfil academico del puesto o "N/A" si no se encuentra.
  """
  # Creo una lista con las palabras clave que indican la formacion academica
  keywords = ['bachelor', 'licenciatura', 'ingenieria', 'computer science', 'software engineering', 'master', 'degree']
  # Devuelvo "Ingeniería/Licenciatura en áreas afines" si se encuentra alguna keyword, y si no, retorno un NA
  return "Ingeniería/Licenciatura en áreas afines" if any(w in text for w in keywords) else "N/A"

# Funcion "Principal"
def get_job():
  """
    Esta funcion realiza la búsqueda de trabajos usando SerpApi y extrae la información relevante.
    Args: Ninguno, usa las variables globales POSITION, LOCATION, COUNTRY_CODE y MAX_RESULTS.
    Returns:
        list of dict: Lista de trabajos con información como ciudad, empresa, skills, salario, link, etc.
    """
  # Primero, voy a configurar el dominio de la pagina de google y el lenguaje segun sea el pais
  config = { # Intento obtener una tuplca de configuracion por COUNTRY_CODE
      "mx": ("google.com.mx", "es", "mx", "Mexico"),
      "us": ("google.com", "en", "us", "United States")
  }.get(COUNTRY_CODE), ("google.com.mx", "en", "us", "United States") # si no lo encuentra, usa la tupla por defecto a la derecha

  # Desempaqueta: si .get(COUNTRY_CODE) devolvió algo, usa esa tupla; si no, la de fallback.
  google_domain, hl, gl, country_name = config[0] if config[0] else config[1]

  # Defino los parametros para SerpAPI
  params = {
      "api_key": SERPAPI_KEY,
      "engine": "google_jobs",
      "q": POSITION,
      "location": LOCATION,
      "google_domain": google_domain,
      "hl": hl, # idioma del interfaz
      "gl": gl, # geolocalizacion o pais
      "num": MAX_RESULTS,
  }

  jobs = [] # creo una variable llamada jobs para guardar los resultados de los puestos de trabajo

  # Utilizo un try catch para el manejo de errores en caso de que la llamada a la API falle
  try:
    search = GoogleSearch(params) # reo el cliente SerpAPI con los parametros que defini y luego ejecuto la busqueda y obtengo los resultados en un JSON (dict)
    results = search.get_dict()

    # Uteri cada trabajo en "jobs_results" o una lista vacia si no esta, limiado a la cantidad maxima de resultados
    for job in results.get("jobs_results", [])[:MAX_RESULTS]:
      title = job.get("title", "N/A")
      company_name = job.get("company_name", "N/A")
      location = job.get("location", "N/A")
      description = job.get("description", "N/A")
      url = get_job_urls(job, COUNTRY_CODE)
      # Extraigo el salario y desde detected_extension.salary (si existe)
      salary = extract_salary(
          job.get("detected_extensions", {}).get("salary")
      )

      # En esta parte, voy a separar la localizacion del puesto en 3 partes, ciudad, estado y el pais
      parts = [p.strip() for p in location.split(",")] # Los divido por comas y recorto los espacios
      city = parts[0] if len(parts) > 0 else "N/A" # Saco el primer segmento como ciudad si existe
      state = parts[1] if len(parts) > 1 else "N/A" # Saco el segundo segmento como estado si existe
      # Si hay 3+ segmentos, último es país; si no, usa el de config.
      country = parts[-1] if len(parts) > 2 else country_name # Use country_name from config

      text = f"{title} {description}".lower() # concateno el titulo y la descripcion y luego lo convierto a minusculas
      # Creo variables para las habilidades tecnicas y blandas y luego mando a llamar la funcion para extraer dichas habilidades
      tech_skills, soft_skills = extract_skills(text, KEYWORDS_TECHNICAL, KEYWORDS_SOFT)

      # Agrego un dict toda a informacion con el formato requerido para la tarea
      jobs.append({
          "City": city,
          "State": state,
          "Country": country,
          "Company": company_name,
          "Position": title,
          "Academic Profile": get_academic_profile(text),
          "English/Bilingue": get_lenguage(text),
          "Technical Skills": ", ".join(tech_skills) if tech_skills else "N/A",
          "Soft Skills": ", ".join(soft_skills) if soft_skills else "N/A",
          "Salary": salary,
          "Source": job.get("via", "SerpAPI"), # Fuente que indica SerpI
          "Link": url,
      })
  except Exception as e:
    print(e)

  return jobs # Devuelvo la lista de trabajos recopilados

# Ejecucion
jobs_data = get_job() # Obtiene los trabajos y luego los guarda en un libro de excel
if jobs_data:
  save_to_excel(jobs_data)
else:
  print("No se encontraron trabajos para este puesto de trabajo")

Los datos se han guardado en el archivo: resultados_front end developer_Mexico.xlsx
